In [1]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torch.utils.data import WeightedRandomSampler
import torch
import numpy as np
import random
import os

class CustomImageFolder(torch.utils.data.Dataset):
    def __init__(self, root, transform=None, is_valid_file=None):
     self.dataset = datasets.ImageFolder(root, is_valid_file=is_valid_file)
     self.transform = transform
     self.targets = self.dataset.targets

    def __getitem__(self, index):
     image, label = self.dataset[index]
     if self.transform:
         image = self.transform(image=np.array(image))["image"]
     return image, label

    def __len__(self):
     return len(self.dataset)

def extract_patient_ids(filename):
    patient_id = filename.split('_')[0].replace("person", "")
    return patient_id

def split_file_names(input_folder, val_split_perc):
    # Pneumonia files contain patient id, so we group split them by patient to avoid data leakage
    pneumonia_patient_ids = set([extract_patient_ids(fn) for fn in os.listdir(os.path.join(input_folder, 'PNEUMONIA')) if fn.lower().endswith(('.jpeg', '.jpg', '.png'))])
    pneumonia_val_patient_ids = random.sample(list(pneumonia_patient_ids), int(val_split_perc * len(pneumonia_patient_ids)))

    pneumonia_val_filenames = []
    pneumonia_train_filenames = []

    for filename in os.listdir(os.path.join(input_folder, 'PNEUMONIA')):
        if not filename.lower().endswith(('.jpeg', '.jpg', '.png')):
            continue
        patient_id = extract_patient_ids(filename)
        if patient_id in pneumonia_val_patient_ids:
            pneumonia_val_filenames.append(os.path.join(input_folder, 'PNEUMONIA', filename))
        else:
            pneumonia_train_filenames.append(os.path.join(input_folder, 'PNEUMONIA', filename))

    # Normal (by file, no patient information in file names)
    normal_filenames  = [os.path.join(input_folder, 'NORMAL', fn) for fn in os.listdir(os.path.join(input_folder, 'NORMAL')) if fn.lower().endswith(('.jpeg', '.jpg', '.png'))]
    normal_val_filenames = random.sample(normal_filenames, int(val_split_perc * len(normal_filenames)))
    normal_train_filenames = list(set(normal_filenames)-set(normal_val_filenames))

    train_filenames = pneumonia_train_filenames + normal_train_filenames
    val_filenames = pneumonia_val_filenames + normal_val_filenames

    return train_filenames, val_filenames



def create_dataloaders(
    train_dir : str,
    test_dir : str,
    transform : transforms.Compose,
    batch_size : int,
    num_workers : int,
    sampler = False
) -> list[DataLoader, DataLoader]:
    '''
    Function for creating dataloaders
    Args:
        train_dir (str) : dir of training data
        test_dir (str) : dir of test data
        transform (transforms.Compose) : transform to perform on data
        batch_size (int) : Number of samples per batch
        num_workers (int) : number of workers per dataloader
    '''

    # Using Imagefolder to load images
    # Images in seperate folders, one folder for each label
    train_data = datasets.ImageFolder(root = train_dir,
                                      transform = transform
                                      )
    test_data = datasets.ImageFolder(root = test_dir,
                                     transform = transform
                                     )

    # Load into data loadsers
    # If using weighted sampler
    if sampler == True:

        targets = torch.tensor(train_data.targets)  # Get the labels from the dataset
        class_counts = torch.bincount(targets)   # Count how many samples per class
        class_weights = 1.0 / class_counts.float()  # Inverse of the counts gives weights for each class


        sample_weights = class_weights[targets]  # Create weights for each sample

        # Create the WeightedRandomSampler
        sampler = WeightedRandomSampler(weights=sample_weights, num_samples=len(sample_weights), replacement=True)
        train_dataloader = DataLoader(train_data,
                                    batch_size=batch_size,
                                    num_workers = num_workers,
                                    pin_memory = True,
                                    sampler= sampler # Pin memory is use to allow easier transfer to gpu
                                    )
    else:
        train_dataloader = DataLoader(train_data,
                                    batch_size=batch_size,
                                    num_workers = num_workers,
                                    pin_memory = True,
                                    shuffle=True # Pin memory is use to allow easier transfer to gpu
                                    )
    test_dataloader = DataLoader(test_data,
                                 batch_size=batch_size,
                                 shuffle = True,
                                 num_workers = num_workers,
                                 pin_memory = True
                                 )

    return train_dataloader, test_dataloader



def create_dataloaders_with_validation(
    train_dir : str,
    test_dir : str,
    train_transform : transforms.Compose,
    test_transform,
    batch_size : int,
    num_workers : int,
    sampler = False
) -> list[DataLoader, DataLoader]:
    '''
    Function for creating dataloaders
    Args:
        train_dir (str) : dir of training data
        test_dir (str) : dir of test data
        transform (transforms.Compose) : transform to perform on data
        batch_size (int) : Number of samples per batch
        num_workers (int) : number of workers per dataloader
    '''
    test_filenames, val_file_names = split_file_names(train_dir, 0.2)
    # Using Imagefolder to load images
    # Images in seperate folders, one folder for each label
    train_data = CustomImageFolder(root = train_dir,
                                      transform = train_transform,
                                      is_valid_file=lambda x: x in test_filenames)
    val_data = CustomImageFolder(root = train_dir,
                                    transform=  test_transform,
                                    is_valid_file= lambda x : x in val_file_names)
    test_data = CustomImageFolder(root = test_dir,
                                     transform = test_transform
                                     )

    # Load into data loadsers
    # If using weighted sampler
    if sampler == True:

        targets = torch.tensor(train_data.targets)  # Get the labels from the dataset
        class_counts = torch.bincount(targets)   # Count how many samples per class
        class_weights = 1.0 / class_counts.float()  # Inverse of the counts gives weights for each class


        sample_weights = class_weights[targets]  # Create weights for each sample

        # Create the WeightedRandomSampler
        sampler = WeightedRandomSampler(weights=sample_weights, num_samples=len(sample_weights), replacement=True)
        train_dataloader = DataLoader(train_data,
                                    batch_size=batch_size,
                                    num_workers = num_workers,
                                    pin_memory = True,
                                    sampler= sampler # Pin memory is use to allow easier transfer to gpu
                                    )
    else:
        train_dataloader = DataLoader(train_data,
                                    batch_size=batch_size,
                                    num_workers = num_workers,
                                    pin_memory = True,
                                    shuffle=True # Pin memory is use to allow easier transfer to gpu
                                    )

    test_dataloader = DataLoader(test_data,
                                 batch_size=batch_size,
                                 shuffle = True,
                                 num_workers = num_workers,
                                 pin_memory = True
                                 )

    return train_dataloader, test_dataloader




# Importing again just to write to file
import torch.nn as nn
import torch
from tqdm.auto import tqdm
from typing import Dict, List, Tuple
from sklearn.metrics import f1_score, accuracy_score
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
from torch.utils.data import *
import matplotlib.pyplot as plt



def train_step(
               model: torch.nn.Module,
               dataloader: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               optimizer: torch.optim.Optimizer,
               device: torch.device = DEVICE
               ) -> Tuple[float, float]:
  """
  Trains a PyTorch model for a single epoch.
  Args:
    model: A PyTorch model to be trained.
    dataloader: A DataLoader object.
    loss_fn: A PyTorch loss function.
    optimizer: A PyTorch optimizer.
    device: A target device to compute on (e.g. "cuda" or "cpu").
  Returns:
    A tuple of training loss and training accuracy metrics.
    In the form (train_loss, train_accuracy). For example:
  """
  # Put model in train mode
  model.train()

  # Setup train loss and train accuracy values
  train_loss, train_acc = 0, 0

  # Loop through data loader data batches
  for batch, (X, y) in enumerate(dataloader):
      # Send data to target device
      X, y = X.to(device), y.to(device)

      # 1. Forward pass
      y_pred = model(X)

      # 2. Calculate  and accumulate loss
      loss = loss_fn(y_pred, y)
      train_loss += loss.item()

      # 3. Optimizer zero grad
      optimizer.zero_grad()

      # 4. Loss backward
      loss.backward()

      # 5. Optimizer step
      optimizer.step()

      # Calculate and accumulate accuracy metric across all batches
      y_pred_class = torch.argmax(torch.softmax(y_pred, dim=1), dim=1)
      train_acc += (y_pred_class == y).sum().item()/len(y_pred)

  # Adjust metrics to get average loss and accuracy per batch
  train_loss = train_loss / len(dataloader)
  train_acc = train_acc / len(dataloader)
  return train_loss, train_acc


def test_step(
        model: torch.nn.Module,
              dataloader: torch.utils.data.DataLoader,
              loss_fn: torch.nn.Module,
              device: torch.device) -> Tuple[float, float]:
  """Tests a PyTorch model for a single epoch.

  Turns a target PyTorch model to "eval" mode and then performs
  a forward pass on a testing dataset.

  Args:
    model: A PyTorch model to be tested.
    dataloader: A DataLoader instance for the model to be tested on.
    loss_fn: A PyTorch loss function to calculate loss on the test data.
    device: A target device to compute on (e.g. "cuda" or "cpu").

  Returns:
    A tuple of testing loss and testing accuracy metrics.
    In the form (test_loss, test_accuracy). For example:

    (0.0223, 0.8985)
  """
  # Put model in eval mode
  model.eval()

  # Setup test loss and test accuracy values
  test_loss, test_acc = 0, 0

  # Turn on inference context manager
  with torch.inference_mode():
      # Loop through DataLoader batches
      for batch, (X, y) in enumerate(dataloader):
          # Send data to target device
          X, y = X.to(device), y.to(device)

          # 1. Forward pass
          test_pred = model(X)

          # 2. Calculate and accumulate loss
          loss = loss_fn(test_pred, y)
          test_loss += loss.item()

          # Calculate and accumulate accuracy
          test_pred_labels = test_pred.argmax(dim=1)
          test_acc += ((test_pred_labels == y).sum().item()/len(test_pred_labels))

  # Adjust metrics to get average loss and accuracy per batch
  test_loss = test_loss / len(dataloader)
  test_acc = test_acc / len(dataloader)
  return test_loss, test_acc


def train_model(
        train_data_loader : DataLoader,
        test_data_loader : DataLoader,
        model : nn.Module,
        loss_funcion : nn.Module ,
        epoches : int = 10,
        optim_func : torch.optim = torch.optim.Adam,
        learn_rate : float = 0.001,
        device = DEVICE,
        plot_loss_rates : bool = True
        ) -> torch.nn.Module:
    '''
    Function for fitting a model to a data
    '''
    torch.cuda.empty_cache()

    optimiser = optim_func(model.parameters(), learn_rate)

    results = {"train_loss": [],
      "train_acc": [],
      "test_loss": [],
      "test_acc": []
   }

    train_losses = []
    test_losses = []

    for epoch in range(epoches):
        # Put model in train mode
        train_loss, train_acc = train_step(
           model,
           train_data_loader,
           loss_funcion,
           optimiser,
           device
          )
        test_loss, test_acc = test_step(
           model,
           test_data_loader,
           loss_funcion,
           device
            )
        train_losses.append(train_loss)
        test_losses.append(test_loss)
        print(
            f"Epoch: {epoch+1} | "
            f"train_loss: {train_loss:.4f} | "
            f"train_acc: {train_acc:.4f} | "
            f"test_loss: {test_loss:.4f} | "
            f"test_acc: {test_acc:.4f}"
          )
    if plot_loss_rates:
      plt.figure(figsize=(8,6))
      plt.plot(range(epoches), train_losses)
      plt.plot(range(epoches), test_losses)
      plt.show()


    # Update results dictionary
    results["train_loss"].append(train_loss)
    results["train_acc"].append(train_acc)
    results["test_loss"].append(test_loss)
    results["test_acc"].append(test_acc)

  # Return the filled results at the end of the epochs


def evalutation_model(
      model : nn.Module,
      evaluation_data : DataLoader,
      loss_fn : nn,
      device = DEVICE,
      ):
    '''
    Function for model evalutation, should of made it work with test_step but doesn't matter too much.
    Args:

    Returns:
    '''

    true_labels = []
    predicted_labels = []
    total_loss = 0
    correct = 0
    total = 0
    model.eval().to(device)

    with torch.no_grad():
        for inputs, labels in evaluation_data:

            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)

            loss = loss_fn(outputs, labels)
            total_loss += loss.item()

            _, predicted = torch.max(outputs.data, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

            true_labels.extend(labels.cpu().numpy())
            predicted_labels.extend(predicted.cpu().numpy())

    test_loss = total_loss / len(evaluation_data)
    test_accuracy = correct / total
    print(f'The test f1 score of this model is {f1_score(true_labels, predicted_labels)}')
    print(f'The test accuracy of this model is {test_accuracy}')

    return test_loss, test_accuracy, true_labels, predicted_labels



# Imports

In [2]:
print(os.chdir("./drive/MyDrive/"))

None


In [3]:
from google.colab import drive
drive.mount('/content/drive')

import os
os.environ['KAGGLE_CONFIG_DIR'] = '/content/drive/My Drive/kaggle'
!kaggle datasets download -d paultimothymooney/chest-xray-pneumonia
!unzip -q -o chest-xray-pneumonia.zip

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Dataset URL: https://www.kaggle.com/datasets/paultimothymooney/chest-xray-pneumonia
License(s): other
chest-xray-pneumonia.zip: Skipping, found more recently modified local copy (use --force to force download)


In [4]:
# System imports
import os
import sys
from pathlib import Path
# Plotting imports
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
from matplotlib.pyplot import imread
import seaborn as sn
# Pytorch imports
import torch
import torch.nn as nn
import torch.nn.functional as F
# Vision imports
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import WeightedRandomSampler

# Random
import random
# TQDM
import tqdm
from torchsummary import summary
import gc



# setting up device agnostics
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# Setting number of works and directories for data
NUM_WORKERS = os.cpu_count()
TRAIN_DIR = "kaggle/chest_xray/train/"
TEST_DIR = "kaggle/chest_xray/test/"
DATA_DIR = "kaggle/chest_xray/chest_xray/"




TRAINING_MODEL_NUMBER = 0



data_augmentation_transform = data_transforms_train = transforms.Compose([
        transforms.RandomRotation(20),  # Randomly rotate the image within a range of (-20, 20) degrees
        transforms.RandomHorizontalFlip(p=0.5),  # Randomly flip the image horizontally with 50% probability
        transforms.RandomResizedCrop(size=(256, 256), scale=(0.8, 1.0)),  # Randomly crop the image and resize it
        transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1),  # Randomly change the brightness, contrast, saturation, and hue
        transforms.RandomApply([transforms.RandomAffine(0, translate=(0.1, 0.1))], p=0.5),  # Randomly apply affine transformations with translation
        transforms.RandomApply([transforms.RandomPerspective(distortion_scale=0.2)], p=0.5),  # Randomly apply perspective transformations
        transforms.Resize(size=(256 ,256)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                            std=[0.229, 0.224, 0.225])
    ])


test_data_transform = transforms.Compose([
        transforms.Resize(size=(256, 256)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                            std=[0.229, 0.224, 0.225])
    ])

# Transfer learning

Transfer learning is a powerful tool that can be used to increase the power of a network classifier without the need for the computationally expensive task of training a CNN. Transfer learning works by using a base pre trained model and modifying it for our classifcation task (how you do this depends on the model).#

For this we will use EfficentnetV2 S and will first get a baseline accuracy and f1score for us to improve on. This base line will be run direclty on the unedited 256x256 images.  The effnet model has been adapted for a binary classifacation problem and we will run 3 epoches to train the last layer.

In [5]:
import torchvision.models as models

effnetdefaultmodel = models.efficientnet_v2_m(weights = models.efficientnet_v2_m)
for param in effnetdefaultmodel.parameters():
    param.requires_grad = False
num_features = effnetdefaultmodel.classifier[1].in_features
effnetdefaultmodel.classifier[1] = nn.Linear(num_features, 2)

summary(effnetdefaultmodel, (3, 256, 256), device  = "cpu")

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_V2_M_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_V2_M_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/efficientnet_v2_m-dc08266a.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_v2_m-dc08266a.pth
100%|██████████| 208M/208M [00:01<00:00, 141MB/s]


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 24, 128, 128]             648
       BatchNorm2d-2         [-1, 24, 128, 128]              48
              SiLU-3         [-1, 24, 128, 128]               0
            Conv2d-4         [-1, 24, 128, 128]           5,184
       BatchNorm2d-5         [-1, 24, 128, 128]              48
              SiLU-6         [-1, 24, 128, 128]               0
   StochasticDepth-7         [-1, 24, 128, 128]               0
       FusedMBConv-8         [-1, 24, 128, 128]               0
            Conv2d-9         [-1, 24, 128, 128]           5,184
      BatchNorm2d-10         [-1, 24, 128, 128]              48
             SiLU-11         [-1, 24, 128, 128]               0
  StochasticDepth-12         [-1, 24, 128, 128]               0
      FusedMBConv-13         [-1, 24, 128, 128]               0
           Conv2d-14         [-1, 24, 1

In [6]:
if TRAINING_MODEL_NUMBER == 50:
    gc.collect()
    torch.cuda.empty_cache()
    train_data , test_data = create_dataloaders(TRAIN_DIR, TEST_DIR, test_data_transform, 64, NUM_WORKERS)
    print(DEVICE)

    effnetdefaultmodel.to(DEVICE)

    loss_func = torch.nn.CrossEntropyLoss()

    train_model(train_data, test_data, effnetdefaultmodel, loss_func, epoches = 3)

    evalutation_model(effnetdefaultmodel, test_data, loss_func)


Base line mode got the following statistics:
* F1_Score = 86%
* Accuracy = 81%

## Data augmentation + Data balancing

We will now train the final layer using both data augmentation and data balancing to see what effect that has on its accuracy.

For this I will use the same model - for arguments sake I don't think the 3 epoches trained on the un augmentad data will give it any performance boost and I dont want to have to load it again.

In [7]:
if TRAINING_MODEL_NUMBER == 51:
    gc.collect()
    torch.cuda.empty_cache()
    _ , test_data = create_dataloaders(TRAIN_DIR, TEST_DIR, test_data_transform, 64, NUM_WORKERS)
    train_data, _ = create_dataloaders(TRAIN_DIR, TEST_DIR , data_augmentation_transform, 64, NUM_WORKERS, sampler = True)

    print(DEVICE)

    effnetdefaultmodel.to(DEVICE)

    loss_func = torch.nn.CrossEntropyLoss()

    train_model(train_data, test_data, effnetdefaultmodel, loss_func, epoches = 3)

    evalutation_model(effnetdefaultmodel, test_data, loss_func)


Massive improvements just from balancing and data augmentation alone, 7% accuracy increase.

* f1score - 90%
* Accuracy - 87%

## Data augmentation + balance load  + 224x 224 image size

Just going to look at the effect of creasing image size to 224, if it doesnt increase we will stick with 256.

In [8]:

data_augmentation_transform_224 = data_transforms_train = transforms.Compose([
        transforms.RandomRotation(20),  # Randomly rotate the image within a range of (-20, 20) degrees
        transforms.RandomHorizontalFlip(p=0.5),  # Randomly flip the image horizontally with 50% probability
        transforms.RandomResizedCrop(size=(224, 224), scale=(0.8, 1.0)),  # Randomly crop the image and resize it
        transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1),  # Randomly change the brightness, contrast, saturation, and hue
        transforms.RandomApply([transforms.RandomAffine(0, translate=(0.1, 0.1))], p=0.5),  # Randomly apply affine transformations with translation
        transforms.RandomApply([transforms.RandomPerspective(distortion_scale=0.2)], p=0.5),  # Randomly apply perspective transformations
        transforms.Resize(size=(224 ,224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                            std=[0.229, 0.224, 0.225])
    ])


test_data_transform_224 = transforms.Compose([
        transforms.Resize(size=(224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                            std=[0.229, 0.224, 0.225])
    ])

if TRAINING_MODEL_NUMBER == 52:
    gc.collect()
    torch.cuda.empty_cache()
    _ , test_data = create_dataloaders(TRAIN_DIR, TEST_DIR, test_data_transform_224, 64, NUM_WORKERS)
    train_data, _ = create_dataloaders(TRAIN_DIR, TEST_DIR , data_augmentation_transform_224, 64, NUM_WORKERS, sampler = True)

    print(DEVICE)

    effnetdefaultmodel.to(DEVICE)

    loss_func = torch.nn.CrossEntropyLoss()

    train_model(train_data, test_data, effnetdefaultmodel, loss_func, epoches = 3)

    evalutation_model(effnetdefaultmodel, test_data, loss_func)


Accuracy and f1 score decrease from chanign image size, so will stick with 256.

## Increasing training time

We will now train back on our 256x256 images and train it for longer increasing it to 10 epoches instead.

In [9]:
if TRAINING_MODEL_NUMBER == 53:
    gc.collect()
    torch.cuda.empty_cache()
    _ , test_data = create_dataloaders(TRAIN_DIR, TEST_DIR, test_data_transform, 64, NUM_WORKERS)
    train_data, _ = create_dataloaders(TRAIN_DIR, TEST_DIR , data_augmentation_transform, 64, NUM_WORKERS, sampler = True)

    print(DEVICE)

    effnetdefaultmodel.to(DEVICE)

    loss_func = torch.nn.CrossEntropyLoss()

    train_model(train_data, test_data, effnetdefaultmodel, loss_func, epoches = 10)

    evalutation_model(effnetdefaultmodel, test_data, loss_func)


Slight increase in performacne from the longer training.

* F1score - 90%
* Accuracy - 87%

This is a 7% accuracy increase over the base model.

In [ ]:
import albumentations as A
import cv2 as cv
from albumentations.pytorch import ToTensorV2


data_augmentation_transform = A.Compose([
     A.Rotate(limit=20),
     A.HorizontalFlip(p=0.5),
     A.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1, p=1),
     A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0, rotate_limit=0, p=0.5),
     A.Perspective(scale=(0.05, 0.15), keep_size=True, p=0.5),
     A.Resize(height=256, width=256),
     A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
     ToTensorV2()
 ])

test_data_transform = A.Compose([
        A.Resize(height= 256, width=256),
        A.Normalize(mean=[0.485, 0.456, 0.406],
                            std=[0.229, 0.224, 0.225]),
        ToTensorV2()
    ])

TRAINING_MODEL_NUMBER = 54
if TRAINING_MODEL_NUMBER == 54:


    torch.cuda.empty_cache()
    train_data , test_data = create_dataloaders_with_validation(TRAIN_DIR, TEST_DIR,data_augmentation_transform ,test_data_transform, 256, NUM_WORKERS)

    print(DEVICE)

    effnetdefaultmodel.to(DEVICE)

    loss_func = torch.nn.CrossEntropyLoss()

    train_model(train_data, test_data, effnetdefaultmodel, loss_func, epoches = 10)

    evalutation_model(effnetdefaultmodel, test_data, loss_func)



/usr/local/lib/python3.10/dist-packages/albumentations/__init__.py:13: UserWarning: A new version of Albumentations is available: 1.4.18 (you have 1.4.15). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


cuda
Epoch: 1 | train_loss: 0.5242 | train_acc: 0.7611 | test_loss: 0.5365 | test_acc: 0.7857
Epoch: 2 | train_loss: 0.4008 | train_acc: 0.8331 | test_loss: 0.4528 | test_acc: 0.7892
Epoch: 3 | train_loss: 0.3505 | train_acc: 0.8650 | test_loss: 0.4246 | test_acc: 0.8093
